In [1]:
import tensorflow as tf 

/Users/shinoda/.pyenv/versions/anaconda3-2019.10/envs/tf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/shinoda/.pyenv/versions/anaconda3-2019.10/envs/tf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/shinoda/.pyenv/versions/anaconda3-2019.10/envs/tf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint

# 問題1】スクラッチを振り返る

ここまでのスクラッチを振り返り、ディープラーニングを実装するためにはどのようなものが必要だったかを列挙してください。

バイアスを初期化する必要

活性化関数が必要

ノード必要

# 【問題2】スクラッチとTensorFlowの対応を考える

In [2]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))#tf.nn.softmax#########
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))#tf.sign(tf.nn.softmax(logits) - 0.5)
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, total_loss, val_loss, total_acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 0, loss : 2.7512, val_loss : 20.9034, acc : 0.054, val_acc : 0.375
Epoch 1, loss : 0.1761, val_loss : 0.8709, acc : 0.080, val_acc : 0.812
Epoch 2, loss : 0.0552, val_loss : 0.5479, acc : 0.100, val_acc : 0.812
Epoch 3, loss : 0.1088, val_loss : 0.1460, acc : 0.094, val_acc : 0.938
Epoch 4, loss : 0.1592, val_loss : 0.0043, acc : 0.092, val_acc : 1.000
Epoch 5, loss : 0.4702, val_loss : 0.9513, acc : 0.084, val_acc : 0.938
Epoch 6, loss : 0.8434, val_loss : 10.4279, acc : 0.073, val_acc : 0.625
Epoch 7, loss : 1.1197, val_loss : 1.3854, acc : 0.067, val_acc : 0.812
Epoch 8, loss : 0.1491, val_loss : 0.9312, acc : 0.095, val_acc : 0.938
Epoch 9, loss : 0.0474, val_loss : 0.0112, acc : 0.105, val_acc : 1.000
test_acc : 0.900


In [3]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip list

Package              Version            
-------------------- -------------------
absl-py              0.9.0              
appnope              0.1.0              
astor                0.8.1              
attrs                19.3.0             
backcall             0.1.0              
bleach               3.1.0              
certifi              2019.11.28         
decorator            4.4.1              
defusedxml           0.6.0              
entrypoints          0.3                
gast                 0.3.3              
google-pasta         0.1.8              
grpcio               1.27.2             
h5py                 2.10.0             
importlib-metadata   1.5.0              
ipykernel            5.1.4              
ipython              7.12.0             
ipython-genutils     0.2.0              
jedi                 0.16.0             
Jinja2               2.11.1             
joblib               0.14.1             
jsonschema           3.2.0              
jupyter-client  

＊バイアスを初期化する必要は以下のコードでしている

In [5]:
"""
# variableの初期化
init = tf.global_variables_initializer()
"""

'\n# variableの初期化\ninit = tf.global_variables_initializer()\n'

＊活性化関数が必要は以下のコードでしている

In [10]:
"""
tf.nn.sigmoid_cross_entropy_with_logits
"""

'\ntf.nn.sigmoid_cross_entropy_with_logits\n'

＊ノード必要は以下のコードでしている

In [12]:
"""
n_hidden1 = 50
n_hidden2 = 100
"""

'\nn_hidden1 = 50\nn_hidden2 = 100\n'

# 【問題3】3種類全ての目的変数を使用したIrisのモデルを作成

Irisデータセットのtrain.csvの中で、目的変数Speciesに含まれる3種類全てを分類できるモデルを作成してください。

2クラスの分類と3クラス以上の分類の違いを考慮してください。それがTensorFlowでどのように書き換えられるかを公式ドキュメントなどを参考に調べてください。

sclar = StandardScaler()


node 100,100

loss tf.reduce_mean

opti   Gradient　問い３のcrossentropy 一気にロスを計算する方法がわからない

In [13]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
# データセットの読み込み
dataset_path2 ="Iris.csv"
df_2 = pd.read_csv(dataset_path2)
#print(df_2)
# データフレームから条件抽出
y_2 = df_2["Species"]
#print(y_2)
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_one_hot = enc.fit_transform(y_2[:, np.newaxis])
#y_test_one_hot = enc.transform(y_test[:, np.newaxis])
X_2 = df_2.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
#print(y_one_hot)
#y = np.array(y)
X_2 = np.array(X_2)
# ラベルを数値に変換
#y[y=='Iris-versicolor'] = 0
#y[y=='Iris-virginica'] = 1
#y = y.astype(np.int)[:, np.newaxis]
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X_2, y_one_hot, test_size=0.25, random_state=0)
print(y_train.shape)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=0)
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))#tf.nn.softmax#########
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
#correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.nn.softmax(logits) - 0.5))#tf.sign(tf.nn.softmax(logits) - 0.5)
correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(logits, 1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, total_loss, val_loss, total_acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

(112, 3)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Epoch 0, loss : 2.9904, val_loss : 17.8956, acc : 0.038, val_acc : 0.321
Epoch 1, loss : 0.3314, val_loss : 5.2776, acc : 0.078, val_acc : 0.714
Epoch 2, loss : 0.2252, val_loss : 0.0447, acc : 0.093, val_acc : 0.964
Epoch 3, loss : 0.0811, val_loss : 0.0001, acc : 0.100, val_acc : 1.000
Epoch 4, loss : 0.0139, val_loss : 1.1122, acc : 0.104, val_acc : 0.929
Epoch 5, loss : 0.0583, val_loss : 0.2128, acc : 0.104, val_acc : 0.964
Epoch 6, loss : 0.1301, val_loss : 3.2106, acc : 0.098, val_acc : 0.786
Epoch 7, loss : 0.0921, val_loss : 0.0069, acc : 0.101, val_acc : 1.000
Epoch 8, loss : 0.0802, val_loss : 0.0000, acc : 0.100, val_acc : 1.000
Epoch 9, loss : 0.0437, val_loss : 2.3685, acc : 0.104, val_acc : 0.893
test_acc : 0.921


# 【問題4】House Pricesのモデルを作成

In [28]:
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_boston

In [29]:
df_4 = pd.read_csv("../train.csv")
df_se = df_4.loc[:, ["GrLivArea", "YearBuilt", "SalePrice"]]
df_se.head()      

,GrLivArea,YearBuilt,SalePrice
0,1710,2003,208500
1,1262,1976,181500
2,1786,2001,223500
3,1717,1915,140000
4,2198,2000,250000


In [65]:
#学習データ準備
f_num = (df_se.shape[1] - 1)
train_X_h = np.array(df_se.iloc[:,:f_num])
print(train_X_h.shape)
train_Y_h =np.array(df_se.iloc[:,f_num : f_num + 1])
print(train_Y_h.shape)
n_samples = train_X_h.shape[0]
print(n_samples)
#正規化
ss = StandardScaler()
ss.fit(train_X_h)
train_X_h = ss.transform(train_X_h)
train_X_h

(1460, 2)
(1460, 1)
1460


array([[ 0.37033344,  1.05099379],
       [-0.48251191,  0.15673371],
       [ 0.51501256,  0.9847523 ],
       ...,
       [ 1.56964722, -1.00249232],
       [-0.83278768, -0.70440562],
       [-0.49393395, -0.20759447]])

In [66]:
#プレースホルダー
with tf.name_scope('data'):
    X = tf.placeholder(tf.float32, shape = [None, f_num ], name = "X")
    Y = tf.placeholder(tf.float32, name = "Y")
#変数（パラメータ）
with tf.name_scope('parameter'):
    W = tf.Variable(tf.zeros([f_num, 1]), name = "weight")
    b = tf.Variable(tf.zeros([1]), name = "bias")

In [67]:
#モデル
with tf.name_scope('model'):
    pred = tf.add(tf.matmul(X, W), b) #matmulは行列の積

In [68]:
#損失関数
with tf.name_scope('loss'):
    # Mean squared error
    loss = tf.reduce_mean(tf.square(pred - Y))
    tf.summary.scalar('loss', loss)
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.01)
train_step = optimizer.minimize(loss)
#決定係数(R2)
with tf.name_scope('r2'):
    r2 = 1 - (tf.reduce_sum(tf.square(Y - pred)) / tf.reduce_sum(tf.square(Y - tf.reduce_mean(Y))))
    tf.summary.scalar('r2', r2)

In [71]:
with tf.Session() as sess:
    # ログの設定
    summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("boston_log", sess.graph)
    
    sess.run(tf.global_variables_initializer())#変数初期化
    
    for i in range(1000):
        sess.run(train_step, feed_dict={X: train_X_h, Y: train_Y_h})
        if i != 0 and i % 50 == 0: # 50ステップごとに精度を出力
            train_summary, train_loss, train_r2 = sess.run([summary, loss, r2], feed_dict={X: train_X_h, Y:train_Y_h})# コストと精度を出力
            writer.add_summary(train_summary, i) #summaryの更新
            
            print("Step:", '%04d' % (i), "loss=", "{:.9f}".format(train_loss), "r2=", "{:.9f}".format(train_r2), "W=", sess.run(W), "b=", sess.run(b))            
            
    training_cost, training_r2 = sess.run([loss,r2], feed_dict={X: train_X_h, Y: train_Y_h})
    print("Training cost=", training_cost, "Training r2=", training_r2, "W=", sess.run(W), "b=", sess.run(b), '\n')


Step: 0050 loss= 6712870912.000000000 r2= -0.064388037 W= [[34123.766]
 [23790.438]] b= [116352.91]
Step: 0100 loss= 2769923072.000000000 r2= 0.560802937 W= [[44675.367]
 [29865.111]] b= [157407.42]
Step: 0150 loss= 2259183872.000000000 r2= 0.641785383 W= [[48143.69 ]
 [31336.791]] b= [172358.22]
Step: 0200 loss= 2192371456.000000000 r2= 0.652379155 W= [[49322.8  ]
 [31625.441]] b= [177802.8]
Step: 0250 loss= 2183561728.000000000 r2= 0.653775990 W= [[49739.426]
 [31644.932]] b= [179785.53]
Step: 0300 loss= 2182391552.000000000 r2= 0.653961539 W= [[49892.777]
 [31621.164]] b= [180507.61]
Step: 0350 loss= 2182235392.000000000 r2= 0.653986335 W= [[49951.51]
 [31600.92]] b= [180770.6]
Step: 0400 loss= 2182214400.000000000 r2= 0.653989673 W= [[49974.84 ]
 [31589.023]] b= [180866.34]
Step: 0450 loss= 2182211072.000000000 r2= 0.653990149 W= [[49984.395]
 [31582.863]] b= [180901.23]
Step: 0500 loss= 2182210560.000000000 r2= 0.653990269 W= [[49988.406]
 [31579.867]] b= [180913.92]
Step: 0550 lo

# 【問題5】MNISTのモデルを作成

ニューラルネットワークのスクラッチで使用したMNISTを分類するモデルを作成してください。


3クラス以上の分類という点ではひとつ前のIrisと同様です。入力が画像であるという点で異なります。


スクラッチで実装したモデルの再現を目指してください。

In [123]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf

# 重み変数
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

# バイアス変数
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# 畳み込み
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

# プーリング
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

def main(_):
    # データ取得
    mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
    
    print(mnist)
    # placeholder作成
    x = tf.placeholder(tf.float32, [None, 784])
    y_ = tf.placeholder(tf.float32, [None, 10])

    # 畳み込み１層目
    W_conv1 = weight_variable([5, 5, 1, 32])
    b_conv1 = bias_variable([32])
    x_image = tf.reshape(x, [-1,28,28,1])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)

    # 畳み込み２層目
    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)

    # 全結合層
    W_fc1 = weight_variable([7 * 7 * 64, 1024])
    b_fc1 = bias_variable([1024])
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

    # ドロップアウト層
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    # 出力層　　　　　　　　　
    W_fc2 = weight_variable([1024, 10])
    b_fc2 = bias_variable([10])
    y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
    

    # 損失関数（交差エントロピー誤差）
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))#labels=y_conv, logits=y_
    
    # 勾配
    train_step = tf.train.AdamOptimizer(learning_rate=1e-5).minimize(cross_entropy)

    # 精度
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    # セッション
    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())

  # トレーニング
    for i in range(5000):
        batch = mnist.train.next_batch(50)

        if i % 500 == 0:
          # 途中経過（500件ごと）
            train_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
            print("step %d, training accuracy %f" % (i, train_accuracy))

        # トレーニング実行
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

# 評価
    print("test accuracy %f" % accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--data_dir', type=str, default='/tmp/tensorflow/mnist/input_data',
                      help='Directory for storing input data')
    FLAGS, unparsed = parser.parse_known_args()
    tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x16b68c310>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x178b93490>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x184ced550>)


/Users/shinoda/.pyenv/versions/anaconda3-2019.10/envs/tf/lib/python3.7/site-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


step 0, training accuracy 0.140000
step 500, training accuracy 0.840000
step 1000, training accuracy 0.860000
step 1500, training accuracy 0.900000
step 2000, training accuracy 0.900000
step 2500, training accuracy 0.920000
step 3000, training accuracy 0.940000


KeyboardInterrupt: 